In [1]:
import os
import random
from distributed import Client, LocalCluster

from helpers import fake_io, n_fibonacci, digits_fibonacci

In [3]:
n_cpu = os.cpu_count()
n_cpu

4

In [4]:
cluster45 = LocalCluster(n_workers=n_cpu, threads_per_worker=5)
cluster4 = LocalCluster()

cluster45, cluster4

(LocalCluster('tcp://127.0.0.1:41457', workers=4, ncores=20),
 LocalCluster('tcp://127.0.0.1:39232', workers=4, ncores=4))

In [5]:
client45 = Client(cluster45)
client45

Client Scheduler: tcp://127.0.0.1:41457 Dashboard: http://127.0.0.1:8787,Cluster Workers: 4 Cores: 20 Memory: 19.93 GB


In [6]:
client4 = Client(cluster4)
client4

Client Scheduler: tcp://127.0.0.1:39232 Dashboard: http://127.0.0.1:37878,Cluster Workers: 4 Cores: 4 Memory: 4.98 GB


In [8]:
# input_numbers = [
#     int(2E5) + random.randint(-1E5, 1E5) // 2 for __ in range(16)
# ]
random_input = [
    int(2E5) + random.randint(-1E5, 1E5) // 2 for __ in range(32)
]
print(list(random_input))

[171319, 198520, 196566, 177460, 237841, 184993, 241988, 239593, 234939, 159358, 229840, 233227, 172431, 220010, 245568, 182087, 171417, 210589, 199735, 185382, 233886, 240198, 223000, 178785, 239733, 192504, 157331, 180021, 154207, 191347, 202581, 185093]


In [9]:
def process_datum(n):
    m = fake_io(n) # e.g read data from disk/API
    n_digits = digits_fibonacci(m)
    return fake_io(n_digits) # e.g. write data to disk/API

In [10]:
%%time
list(map(process_datum, input_numbers[:8]))

CPU times: user 5.05 s, sys: 0 ns, total: 5.05 s
Wall time: 9.05 s


[47490, 38969, 31526, 52134, 35722, 50472, 42925, 40669]

In [11]:
%%time
f_read = client4.map(fake_io, random_input)
f_proc = client4.map(digits_fibonacci, f_read)
f_out  = client4.map(fake_io, f_proc)
client4.gather(f_out)

#     print(list(f.result() for f in f_out))

CPU times: user 1.42 s, sys: 260 ms, total: 1.68 s
Wall time: 9.15 s


In [18]:
%%time
f_read = client45.map(fake_io, random_input)
f_proc = client45.map(digits_fibonacci, f_read)
f_out  = client45.map(fake_io, f_proc)
client45.gather(f_out)

#     print(list(f.result() for f in f_out))

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 18.3 ms


In [23]:
%%time
futures = client45.map(digits_fibonacci, random_input)
client45.gather(futures)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 15.7 ms


LocalCluster('tcp://127.0.0.1:41457', workers=4, ncores=20)